# March Madness 2025

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Hypothesis
Each team can be modeled by x hidden features. In each game, these hidden features interact in a nonlinear fashion to determine the outcome of the game

## Preparing the data
Load the data

In [2]:
mens = pd.read_csv('data/MRegularSeasonDetailedResults.csv')
mens['League'] = 'M'
mens.describe()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
count,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,...,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000,116723.000000
mean,2014.051044,70.072462,1288.243422,75.859651,1283.044987,63.857732,0.068658,26.392099,55.746305,7.339085,...,20.138276,12.072488,17.736907,10.480668,21.632934,11.405867,13.737130,5.901031,3.144239,19.324709
std,6.464256,35.845605,105.298971,11.007412,104.764160,10.851210,0.305052,4.683480,7.461328,3.116574,...,6.064958,5.345290,7.085348,4.221941,4.519345,3.724047,4.536147,2.778302,2.628125,4.551727
min,2003.000000,0.000000,1101.000000,34.000000,1101.000000,20.000000,0.000000,10.000000,26.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,4.000000
25%,2009.000000,38.000000,1199.000000,68.000000,1192.000000,57.000000,0.000000,23.000000,51.000000,5.000000,...,16.000000,8.000000,13.000000,7.000000,19.000000,9.000000,11.000000,4.000000,1.000000,16.000000
50%,2014.000000,73.000000,1287.000000,75.000000,1282.000000,64.000000,0.000000,26.000000,55.000000,7.000000,...,20.000000,12.000000,17.000000,10.000000,21.000000,11.000000,13.000000,6.000000,3.000000,19.000000
75%,2020.000000,101.000000,1381.000000,83.000000,1374.000000,71.000000,0.000000,29.000000,60.000000,9.000000,...,24.000000,15.000000,22.000000,13.000000,25.000000,14.000000,17.000000,8.000000,4.000000,22.000000
max,2025.000000,132.000000,1480.000000,149.000000,1480.000000,144.000000,6.000000,57.000000,103.000000,26.000000,...,59.000000,48.000000,65.000000,36.000000,49.000000,31.000000,41.000000,22.000000,33.000000,45.000000


In [3]:
womens = pd.read_csv('data/WRegularSeasonDetailedResults.csv')
womens['League'] = 'W'
womens.describe()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
count,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,...,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000,79639.000000
mean,2017.310476,68.860759,3285.050867,71.711963,3286.594658,57.234370,0.051583,25.847537,58.980010,6.268876,...,17.913974,10.511119,15.515175,11.395447,22.441116,10.933688,16.745024,6.923831,3.434950,18.204184
std,4.531798,36.258086,104.022507,11.547894,105.457243,10.964583,0.258755,4.982451,7.975729,3.125925,...,6.469817,4.938106,6.632564,4.639725,4.939763,3.805204,5.597689,3.279905,3.666537,4.557235
min,2010.000000,0.000000,3101.000000,30.000000,3101.000000,11.000000,0.000000,9.000000,30.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,2013.000000,35.000000,3196.000000,64.000000,3195.000000,50.000000,0.000000,22.000000,53.000000,4.000000,...,13.000000,7.000000,11.000000,8.000000,19.000000,8.000000,13.000000,5.000000,1.000000,15.000000
50%,2017.000000,72.000000,3283.000000,71.000000,3287.000000,57.000000,0.000000,25.000000,59.000000,6.000000,...,17.000000,10.000000,15.000000,11.000000,22.000000,11.000000,16.000000,7.000000,3.000000,18.000000
75%,2021.000000,101.000000,3376.000000,79.000000,3377.000000,64.000000,0.000000,29.000000,64.000000,8.000000,...,22.000000,14.000000,20.000000,14.000000,26.000000,13.000000,20.000000,9.000000,4.000000,21.000000
max,2025.000000,132.000000,3480.000000,140.000000,3480.000000,130.000000,5.000000,58.000000,113.000000,30.000000,...,80.000000,37.000000,52.000000,38.000000,53.000000,34.000000,49.000000,26.000000,42.000000,47.000000


The IDs are definitely distinct so we can combine into a single dataframe

In [4]:
data = pd.concat([mens, womens])

Get the distinct team/Years

In [5]:
teams = pd.concat([data[['WTeamID', 'Season', 'League']].rename(columns={'WTeamID': 'TeamID'}),
                   data[['LTeamID', 'Season', 'League']].rename(columns={'LTeamID': 'TeamID'})]).drop_duplicates().reset_index()

Define the training data. The x's will be the indexes of two team IDs, the y's will be 1 if the first team won, 0 otherwise.

In [6]:
teamMapping = {(x.TeamID, x.Season): x.Index for x in teams.itertuples()}

In [7]:
data

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,League
0,2003,10,1104,68,1328,62,N,0,27,58,...,16,22,10,22,8,18,9,2,20,M
1,2003,10,1272,70,1393,63,N,0,26,62,...,9,20,20,25,7,12,8,6,16,M
2,2003,11,1266,73,1437,61,N,0,24,58,...,14,23,31,22,9,12,2,5,23,M
3,2003,11,1296,56,1457,50,N,0,18,38,...,8,15,17,20,9,19,4,3,23,M
4,2003,11,1400,77,1208,71,N,0,30,61,...,17,27,21,15,12,10,7,1,14,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79634,2025,84,3450,65,3333,57,A,0,24,58,...,7,9,11,23,10,8,5,10,14,W
79635,2025,85,3129,89,3307,80,H,0,33,73,...,7,16,4,25,16,4,2,13,13,W
79636,2025,85,3234,85,3321,80,H,0,29,60,...,13,20,6,25,18,8,2,13,21,W
79637,2025,85,3337,55,3258,43,H,0,21,50,...,14,17,11,20,6,8,2,10,13,W


In [8]:
def gen_dataset(data):
    winners = data.apply(lambda x: teamMapping[(x.WTeamID, x.Season)], axis=1)
    losers = data.apply(lambda x: teamMapping[(x.LTeamID, x.Season)], axis=1)
    x_tensor = torch.from_numpy(np.concatenate([np.stack([winners, losers], axis=1), np.stack([losers, winners], axis=1)]))
    y_tensor = torch.from_numpy(np.concatenate([(data.WScore-data.LScore), (data.LScore-data.WScore)]).reshape((-1,1))).double()
    return TensorDataset(x_tensor, y_tensor)

In [9]:
dataset = gen_dataset(data)

Generate the train/validation split

In [10]:
batch_size=100

generator = torch.Generator().manual_seed(20250217)
train_data, validation_data = torch.utils.data.random_split(dataset, [0.9, 0.1], generator=generator)
train_loader = DataLoader(train_data, batch_size=batch_size)
validation_loader = DataLoader(validation_data, batch_size=batch_size)

## The Model
Define the model. Combine the embeddings for the two teams, go to a hidden layer, and then output to a prediction if the first team won

In [11]:
class Model(nn.Module):
    def __init__(self, embedding_size=64, model_size=16, dropout=0.1):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(len(teams), embedding_size)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.fc1 = nn.Linear(2*embedding_size, model_size)
        self.score_fc = nn.Linear(model_size, 1)
        self.result_fc = nn.Linear(model_size, 1)
        self.double()

    def forward(self, x):
        team = self.embedding(x[:,0])
        opponent = self.embedding(x[:,1])
        matchup = self.dropout1(torch.cat([team, opponent], axis=1))
        hidden = self.dropout2(F.relu(self.fc1(matchup)))
        score = self.score_fc(hidden)
        result = F.sigmoid(self.result_fc(hidden))
        return score, result
        

In [12]:
model = Model(embedding_size=128, model_size=64, dropout=0.25).to(device)

## Training the model

Define the training function

In [13]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

def train(data, model, loss_fn, optimizer, full_loss=True):
    size = len(data.dataset)
    model.train()
    for batch, (x, y) in enumerate(data):
        x = x.to(device)
        y = y.to(device)
        pred_score, pred_result = model(x)
        actual_result = (y > 0).double()
        score_loss = loss_fn(pred_score, y)
        result_loss = loss_fn(pred_result, actual_result)
        if full_loss:
            (score_loss + 10 * result_loss).backward()
        else:
            result_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            score_loss, result_loss, current = score_loss.item(), result_loss.item(), (batch + 1) * len(x)
            print(f"score loss: {score_loss:>7f}, result loss: {result_loss:>7f} [{current:>6d}/{size:>6d}]", end="\r")

Define the testing function

In [14]:
def test(data, model, loss_fn, label="Test"):
    size = len(data.dataset)
    num_batches = len(data)
    model.eval()
    score_loss, result_loss, correct = 0, 0, 0
    with torch.no_grad():
        for x, y in data:
            x = x.to(device)
            y = y.to(device)
            score_pred, result_pred = model(x)
            actual_result = (y > 0).double()
            score_loss += loss_fn(score_pred, y).item()
            result_loss += loss_fn(result_pred, actual_result)
            correct += ((result_pred >= 0.5) == (actual_result == 1)).type(torch.float).sum().item()
    score_loss /= num_batches
    result_loss /= num_batches
    correct /= size
    print(f"{label}: Accuracy: {(100*correct):>0.1f}%, Score loss: {score_loss:>8f}, Result loss: {result_loss:>8f}")

Train the model

In [15]:
n_epochs = 16
for i in range(n_epochs):
    print(f"Epoch {i}")
    train(train_loader, model, loss_fn, optimizer)
    test(train_loader, model, loss_fn, label="Train")
    test(validation_loader, model, loss_fn, label="Validation")

Epoch 0
Train: Accuracy: 57.1%, Score loss: 263.636632, Result loss: 0.243204
Validation: Accuracy: 56.2%, Score loss: 263.296530, Result loss: 0.244422
Epoch 1
Train: Accuracy: 63.9%, Score loss: 231.516480, Result loss: 0.223652
Validation: Accuracy: 62.6%, Score loss: 235.239862, Result loss: 0.227550
Epoch 2
Train: Accuracy: 68.5%, Score loss: 200.556467, Result loss: 0.203933
Validation: Accuracy: 66.8%, Score loss: 207.398324, Result loss: 0.210133
Epoch 3
Train: Accuracy: 71.6%, Score loss: 175.834077, Result loss: 0.188923
Validation: Accuracy: 69.6%, Score loss: 184.701818, Result loss: 0.196391
Epoch 4
Train: Accuracy: 73.5%, Score loss: 157.989261, Result loss: 0.177771
Validation: Accuracy: 71.6%, Score loss: 167.870227, Result loss: 0.186025
Epoch 5
Train: Accuracy: 74.7%, Score loss: 146.252595, Result loss: 0.171293
Validation: Accuracy: 72.8%, Score loss: 156.618105, Result loss: 0.179574
Epoch 6
Train: Accuracy: 75.4%, Score loss: 138.373359, Result loss: 0.166488
Vali

Fine tune with only the result

In [16]:
for i in range(n_epochs):
    print(f"Epoch {i}")
    train(train_loader, model, loss_fn, optimizer, full_loss=False)
    test(train_loader, model, loss_fn, label="Train")
    test(validation_loader, model, loss_fn, label="Validation")

Epoch 0
Train: Accuracy: 76.6%, Score loss: 119.594737, Result loss: 0.157981
Validation: Accuracy: 75.0%, Score loss: 130.339371, Result loss: 0.166107
Epoch 1
Train: Accuracy: 76.7%, Score loss: 119.579339, Result loss: 0.157899
Validation: Accuracy: 75.0%, Score loss: 130.321221, Result loss: 0.166059
Epoch 2
Train: Accuracy: 76.7%, Score loss: 119.529971, Result loss: 0.157320
Validation: Accuracy: 75.1%, Score loss: 130.256596, Result loss: 0.165770
Epoch 3
Train: Accuracy: 77.1%, Score loss: 119.617322, Result loss: 0.155836
Validation: Accuracy: 75.2%, Score loss: 130.369072, Result loss: 0.165224
Epoch 4
Train: Accuracy: 77.5%, Score loss: 119.957029, Result loss: 0.153474
Validation: Accuracy: 75.3%, Score loss: 130.692715, Result loss: 0.164343
Epoch 5
Train: Accuracy: 77.8%, Score loss: 120.779306, Result loss: 0.151953
Validation: Accuracy: 75.3%, Score loss: 131.585308, Result loss: 0.163952
Epoch 6
Train: Accuracy: 78.1%, Score loss: 122.156921, Result loss: 0.150916
Vali

With this model we can predict the output of about three quarters of regular season games.

## Load the tourney data to test with

In [17]:
mens_tourney = pd.read_csv('data/MNCAATourneyDetailedResults.csv')
womens_tourney = pd.read_csv('data/WNCAATourneyDetailedResults.csv')
tourney = pd.concat([mens_tourney, womens_tourney])

tourney_dataset = gen_dataset(tourney)
tourney_loader = DataLoader(tourney_dataset, batch_size=batch_size)

In [18]:
test(tourney_loader, model, loss_fn, label="Tourney")

Tourney: Accuracy: 72.5%, Score loss: 156.686935, Result loss: 0.178529


When it comes to tournament results we get about 7 out of 10 results. The lower result is likely due to teams having increased pairity.

### Performance by year


In [19]:
for season in tourney.Season.unique():
    loader = DataLoader(gen_dataset(tourney[tourney.Season == season]), batch_size=batch_size)
    test(loader, model, loss_fn, label=f"{season} Tournament")

2003 Tournament: Accuracy: 66.4%, Score loss: 160.445447, Result loss: 0.210222
2004 Tournament: Accuracy: 73.4%, Score loss: 130.199073, Result loss: 0.208641
2005 Tournament: Accuracy: 74.2%, Score loss: 108.259272, Result loss: 0.185513
2006 Tournament: Accuracy: 67.2%, Score loss: 108.850306, Result loss: 0.212178
2007 Tournament: Accuracy: 79.7%, Score loss: 89.687063, Result loss: 0.147681
2008 Tournament: Accuracy: 76.6%, Score loss: 159.348280, Result loss: 0.162999
2009 Tournament: Accuracy: 74.2%, Score loss: 127.202835, Result loss: 0.159879
2010 Tournament: Accuracy: 72.4%, Score loss: 141.909713, Result loss: 0.181241
2011 Tournament: Accuracy: 70.0%, Score loss: 158.337081, Result loss: 0.189246
2012 Tournament: Accuracy: 77.3%, Score loss: 127.862382, Result loss: 0.148412
2013 Tournament: Accuracy: 71.5%, Score loss: 184.030911, Result loss: 0.184701
2014 Tournament: Accuracy: 74.2%, Score loss: 169.804555, Result loss: 0.164973
2015 Tournament: Accuracy: 76.5%, Score l

## Generating the submission file
### Phase 1

Write the results

In [20]:
with open('submission.csv', 'w') as f:
    f.write("ID,Pred\n")
    for season in range(2021, 2025):
        for league in ('M', 'W'):
            teams_to_test = sorted(teams[(teams.Season==season) & (teams.League==league)].TeamID.values)
            matchups = [(t1, t2) for t1 in teams_to_test for t2 in teams_to_test if t1 < t2]
            matchups_tensor = torch.Tensor([(teamMapping[(t1, season)], teamMapping[(t2, season)])
                                     for (t1, t2) in matchups]).int().to(device)
            _, predictions = model(matchups_tensor)
            for (t1, t2), pred in zip(matchups, predictions):
                f.write(f"{season}_{t1.item()}_{t2.item()},{pred.item()}\n")

Two teams canceled their 2021 season due to covid but are still in the sample submission. Add in their results

In [21]:
with open('submission.csv', 'a') as f:
    for missing_team in [3169, 3197]:
        for opponent in teams[(teams.Season==2021) & (teams.League=='W')].TeamID.values:
            if opponent > missing_team:
                f.write(f"2021_{missing_team}_{opponent},0\n")
            else:
                f.write(f"2021_{opponent}_{missing_team},1\n")
    f.write(f"2021_3169_3197,0.5\n")

## Save the model

In [22]:
torch.save(model.state_dict(), 'model.pth')